In [3]:
import os
import hydra
import torch
from omegaconf import OmegaConf
import pathlib
from torch.utils.data import DataLoader
import copy
import random
import wandb
import tqdm
import numpy as np
import shutil
from diffusion_policy.workspace.base_workspace import BaseWorkspace
from diffusion_policy.policy.robomimic_image_policy import RobomimicImagePolicy
from diffusion_policy.dataset.base_dataset import BaseImageDataset
from diffusion_policy.env_runner.base_image_runner import BaseImageRunner
from diffusion_policy.common.checkpoint_util import TopKCheckpointManager
from diffusion_policy.common.json_logger import JsonLogger
from diffusion_policy.common.pytorch_util import dict_apply, optimizer_to
import dill

In [11]:
checkpoint = '/teamspace/studios/this_studio/bc_attacks/diffusion_policy/data/experiments/image/lift_ph/diffusion_policy_cnn/train_0/checkpoints/epoch=0300-test_mean_score=1.000.ckpt'
payload = torch.load(open(checkpoint, 'rb'), pickle_module=dill)
cfg_loaded = payload['cfg']
cfg_loaded.task.dataset.dataset_path = os.path.join('/teamspace/studios/this_studio/bc_attacks/diffusion_policy/', cfg_loaded.task.dataset.dataset_path)
dataset = hydra.utils.instantiate(cfg_loaded.task.dataset)

/teamspace/studios/this_studio/bc_attacks/diffusion_policy/data/robomimic/datasets/lift/ph/image_abs.hdf5
Acquiring lock on cache.
Cache does not exist. Creating!


Loading image data: 100%|██████████| 19332/19332 [01:06<00:00, 289.94it/s]


Saving cache to disk.


In [27]:
# get the first three coordaintes of all the actions
coordinates = []
for i in range(len(dataset)):
    action = dataset[i]['action']
    coordinates.append(action[:, :3].numpy())
coordinates = np.array(coordinates)
print(coordinates.shape)

(8118, 16, 3)


In [36]:
import numpy as np
import plotly.graph_objects as go

# Assuming coordinates is a list of numpy arrays
# coordinates = [np.random.rand(10, 3) for _ in range(5)]  # Example data

# Create a Plotly figure
fig = go.Figure()

# Add each trajectory as a scatter3d trace
plot_range = len(coordinates)
for i in range(plot_range):
    fig.add_trace(go.Scatter3d(
        x=coordinates[i][:, 0],
        y=coordinates[i][:, 1],
        z=coordinates[i][:, 2],
        mode='lines',
        name=f'Trajectory {i+1}'
    ))

# Update the layout for better visualization
fig.update_layout(
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    ),
    title='3D Trajectories'
)
fig.write_html('/teamspace/studios/this_studio/bc_attacks/diffusion_policy/plots/images/3d_trajectories.html')
